In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Human Weight Prediction**
Data wrangling, transfor and analysis. Spark ML model, creating, saving and loading.

Objectives:

* Data web scraping, cleaning and transform data into a dataframe
* Create a simple Linear Regression Model, save and load the SparkML model
* Make predictions using the loaded SparkML model

Acknowledgement: SOCR Data Dinov 020108 HeightsWeights
Link: http://socr.ucla.edu/docs/resources/SOCR_Data/SOCR_Data_Dinov_020108_HeightsWeights.html

In [1]:
# Import pandas as pd
import pandas as pd

# Define path to HTML file
html_path = 'http://socr.ucla.edu/docs/resources/SOCR_Data/SOCR_Data_Dinov_020108_HeightsWeights.html'

# Read tables from HTML file and pass to pandas dataframe
table = pd.read_html(html_path)
df = table[0]
df.head()

,0,1,2
0,Index,Height(Inches),Weight(Pounds)
1,1,65.78331,112.9925
2,2,71.51521,136.4873
3,3,69.39874,153.0269
4,4,68.2166,142.3354


In [2]:
df.drop(index=df.index[0],inplace=True) # Drop first row
del df[0] # Delete Index column
df = df.astype(float) # Change value types (string into float)
df.head()

,1,2
1,65.78331,112.9925
2,71.51521,136.4873
3,69.39874,153.0269
4,68.21660,142.3354
5,67.78781,144.2971


In [3]:
# Metric conversions

# Convert inches to cm
df[1] = 2.54 * df[1]

# Convert pounds to kg
df[2] = df[2] * 0.453592

df = df.round(2) # Round to two decimals
df.head()

,1,2
1,167.09,51.25
2,181.65,61.91
3,176.27,69.41
4,173.27,64.56
5,172.18,65.45


In [4]:
# Install spark
!pip install pyspark
!pip install findspark
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=b90abe192928e5daf81df86533b4041e010738d3500a54b9eda1f81861f22d6c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [5]:
# Start Spark session

# Import pyspark, Spark API for Python 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# Start sesion
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/18 02:16:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Create PySpark DataFrame from Pandas

columns = ['height', 'weight']

dataframe = spark.createDataFrame(df, columns)
dataframe.printSchema()
dataframe.show(5)

root
 |-- height: double (nullable = true)
 |-- weight: double (nullable = true)



+------+------+
|height|weight|
+------+------+
|167.09| 51.25|
|181.65| 61.91|
|176.27| 69.41|
|173.27| 64.56|
|172.18| 65.45|
+------+------+
only showing top 5 rows



In [7]:
# Import Spark ML libraries

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
# Converting data frame columns into feature vectors

assembler = VectorAssembler(
    inputCols=['height'],
    outputCol='features')

data = assembler.transform(dataframe).select('features','weight')
data.show(5)

+--------+------+
|features|weight|
+--------+------+
|[167.09]| 51.25|
|[181.65]| 61.91|
|[176.27]| 69.41|
|[173.27]| 64.56|
|[172.18]| 65.45|
+--------+------+
only showing top 5 rows



In [10]:
# Create and Train model
lr = LinearRegression(featuresCol='features', labelCol='weight', maxIter=100)
lr.setRegParam(0.1)

# Fit the model
lrModel = lr.fit(data)

23/05/18 02:18:44 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/18 02:18:45 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [11]:
# Save the model Human Weight and Height 
lrModel.save('human_height.model')

In [13]:
# Load the model
from pyspark.ml.regression import LinearRegressionModel # LinearRegressionModel to load the model
model = LinearRegressionModel.load('human_height.model')

In [14]:
# Make Prediction

# This function converts a scalar number into a dataframe that can be used by the model to predict.
def predict(weight):
    assembler = VectorAssembler(inputCols=["weight"],outputCol="features")
    data = [[weight,0]]
    columns = ["weight", "height"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','height')
    predictions = model.transform(__)
    predictions.select('prediction').show()

In [16]:
# Predict the weight of person who height is 170 cm
predict(170)

+-----------------+
|       prediction|
+-----------------+
|56.18174659878049|
+-----------------+



In [17]:
# Predict the weight of person who height is 170 cm
predict(150)

+-----------------+
|       prediction|
+-----------------+
|45.37342510832839|
+-----------------+



In [18]:
# Predict the weight of person who height is 170 cm
predict(200)

+-----------------+
|       prediction|
+-----------------+
|72.39422883445864|
+-----------------+

